In [ ]:
import sys
import os
from pathlib import Path
import numpy as np
import pandas as pd
import pickle
from datetime import datetime
import json
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif
from sklearn.preprocessing import PolynomialFeatures
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings('ignore')

print("ライブラリ読み込み完了")


In [ ]:
# データパス設定
data_path = '../output/experiments/lstm_v2_w64_s16/preprocessed'
output_path = '../results/demographics_optimization'

# 出力ディレクトリ作成
os.makedirs(output_path, exist_ok=True)

print(f"データパス: {data_path}")
print(f"出力パス: {output_path}")

# 前処理済みデータの読み込み
print("\nDemographics データ読み込み中...")

# データファイルパス
demo_path = os.path.join(data_path, 'X_demographics_windows.pkl')
label_path = os.path.join(data_path, 'y_windows.pkl')

# データ読み込み
with open(demo_path, 'rb') as f:
    X_demographics_all = pickle.load(f)

with open(label_path, 'rb') as f:
    y_all = pickle.load(f)

# numpy配列に変換
X_demographics_all = np.array(X_demographics_all, dtype=np.float32)
y_all = np.array(y_all, dtype=np.int32)

print(f"全データ形状: {X_demographics_all.shape}")
print(f"ラベル形状: {y_all.shape}")
print(f"クラス数: {len(np.unique(y_all))}")

# 訓練・検証データに分割（8:2）
X_demo_train, X_demo_val, y_train, y_val = train_test_split(
    X_demographics_all, y_all, 
    test_size=0.2, 
    random_state=42,
    stratify=y_all
)

print(f"\n訓練データ: {X_demo_train.shape}")
print(f"検証データ: {X_demo_val.shape}")
print(f"訓練ラベル: {y_train.shape}")
print(f"検証ラベル: {y_val.shape}")


In [ ]:
print("Demographics特徴量最適化を開始...")

# 1. 特徴量重要度分析
print("\n1. 特徴量重要度分析中...")

# F統計量による重要度
f_selector = SelectKBest(score_func=f_classif, k='all')
f_selector.fit(X_demo_train, y_train)
f_scores = f_selector.scores_
f_pvalues = f_selector.pvalues_

# 相互情報量による重要度
mi_scores = mutual_info_classif(X_demo_train, y_train, random_state=42)

# RandomForestによる重要度
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_demo_train, y_train)
rf_importance = rf.feature_importances_

# 結果統合
n_features = X_demo_train.shape[1]
feature_importance = pd.DataFrame({
    'feature_idx': range(n_features),
    'f_score': f_scores,
    'f_pvalue': f_pvalues,
    'mi_score': mi_scores,
    'rf_importance': rf_importance
})

# 正規化スコア
feature_importance['f_score_norm'] = (f_scores - f_scores.min()) / (f_scores.max() - f_scores.min())
feature_importance['mi_score_norm'] = (mi_scores - mi_scores.min()) / (mi_scores.max() - mi_scores.min())
feature_importance['rf_importance_norm'] = rf_importance / rf_importance.max()

# 総合スコア（平均）
feature_importance['combined_score'] = (
    feature_importance['f_score_norm'] + 
    feature_importance['mi_score_norm'] + 
    feature_importance['rf_importance_norm']
) / 3

# 重要度順にソート
feature_importance = feature_importance.sort_values('combined_score', ascending=False)

print("特徴量重要度分析完了")
print(f"最重要特徴量上位10: {feature_importance.head(10)['feature_idx'].tolist()}")

# 2. 特徴量選択最適化
print("\n2. 特徴量選択最適化中...")

# ベースライン性能（全特徴量）
rf_baseline = RandomForestClassifier(n_estimators=100, random_state=42)
baseline_scores = cross_val_score(rf_baseline, X_demo_train, y_train, cv=5, scoring='f1_macro')
baseline_mean = baseline_scores.mean()
baseline_std = baseline_scores.std()

print(f"ベースライン性能: {baseline_mean:.4f} ± {baseline_std:.4f}")

# 特徴量数を変えて最適化
n_features_range = [5, 8, 10, 12, 15, 18, 20]
n_features_range = [k for k in n_features_range if k < X_demo_train.shape[1]]

selection_results = []
best_score = 0
best_k = 0
best_selector = None

for k in n_features_range:
    selector = SelectKBest(score_func=f_classif, k=k)
    X_train_selected = selector.fit_transform(X_demo_train, y_train)
    
    rf = RandomForestClassifier(n_estimators=100, random_state=42)
    scores = cross_val_score(rf, X_train_selected, y_train, cv=5, scoring='f1_macro')
    mean_score = scores.mean()
    std_score = scores.std()
    
    selection_results.append({
        'k': k,
        'mean_score': mean_score,
        'std_score': std_score,
        'selected_features': selector.get_support(indices=True).tolist()
    })
    
    print(f"k={k}: {mean_score:.4f} ± {std_score:.4f}")
    
    if mean_score > best_score:
        best_score = mean_score
        best_k = k
        best_selector = selector

print(f"\n最適特徴量数: {best_k}")
print(f"最適性能: {best_score:.4f}")
print(f"改善度: {best_score - baseline_mean:+.4f}")

# 3. 特徴量エンジニアリング最適化
print("\n3. 特徴量エンジニアリング最適化中...")

# 選択された特徴量を使用
X_train_selected = best_selector.transform(X_demo_train)
X_val_selected = best_selector.transform(X_demo_val)

engineering_results = []
best_eng_score = 0
best_eng_method = 'original'
best_transformer = None

# オリジナル特徴量（ベースライン）
rf_orig = RandomForestClassifier(n_estimators=100, random_state=42)
orig_scores = cross_val_score(rf_orig, X_train_selected, y_train, cv=5, scoring='f1_macro')
orig_mean = orig_scores.mean()
orig_std = orig_scores.std()

engineering_results.append({
    'method': 'original',
    'mean_score': orig_mean,
    'std_score': orig_std,
    'n_features': X_train_selected.shape[1]
})

best_eng_score = orig_mean
print(f"オリジナル特徴量: {orig_mean:.4f} ± {orig_std:.4f}")

# 多項式特徴量
for degree in [2]:  # degree=3は特徴量数が多すぎるため除外
    try:
        poly = PolynomialFeatures(degree=degree, include_bias=False)
        X_train_poly = poly.fit_transform(X_train_selected)
        
        if X_train_poly.shape[1] > 100:
            print(f"多項式特徴量(degree={degree}): 特徴量数が多すぎるためスキップ ({X_train_poly.shape[1]} features)")
            continue
        
        rf_poly = RandomForestClassifier(n_estimators=100, random_state=42)
        poly_scores = cross_val_score(rf_poly, X_train_poly, y_train, cv=5, scoring='f1_macro')
        poly_mean = poly_scores.mean()
        poly_std = poly_scores.std()
        
        engineering_results.append({
            'method': f'polynomial_{degree}',
            'mean_score': poly_mean,
            'std_score': poly_std,
            'n_features': X_train_poly.shape[1]
        })
        
        print(f"多項式特徴量(degree={degree}): {poly_mean:.4f} ± {poly_std:.4f} ({X_train_poly.shape[1]} features)")
        
        if poly_mean > best_eng_score:
            best_eng_score = poly_mean
            best_eng_method = f'polynomial_{degree}'
            best_transformer = poly
            
    except Exception as e:
        print(f"多項式特徴量(degree={degree})でエラー: {e}")

# PCA変換
for n_components in [5, 8, 10, 12]:
    if n_components >= X_train_selected.shape[1]:
        continue
        
    try:
        pca = PCA(n_components=n_components, random_state=42)
        X_train_pca = pca.fit_transform(X_train_selected)
        
        rf_pca = RandomForestClassifier(n_estimators=100, random_state=42)
        pca_scores = cross_val_score(rf_pca, X_train_pca, y_train, cv=5, scoring='f1_macro')
        pca_mean = pca_scores.mean()
        pca_std = pca_scores.std()
        
        engineering_results.append({
            'method': f'pca_{n_components}',
            'mean_score': pca_mean,
            'std_score': pca_std,
            'n_features': n_components
        })
        
        print(f"PCA({n_components}): {pca_mean:.4f} ± {pca_std:.4f}")
        
        if pca_mean > best_eng_score:
            best_eng_score = pca_mean
            best_eng_method = f'pca_{n_components}'
            best_transformer = pca
            
    except Exception as e:
        print(f"PCA({n_components})でエラー: {e}")

print(f"\n最良特徴量エンジニアリング: {best_eng_method}")
print(f"最良性能: {best_eng_score:.4f}")
print(f"改善度: {best_eng_score - orig_mean:+.4f}")


In [ ]:
print("最適化された特徴量を作成中...")

# 最適な特徴量選択を適用
X_train_selected = best_selector.transform(X_demo_train)
X_val_selected = best_selector.transform(X_demo_val)

# 最適な特徴量エンジニアリングを適用
if best_eng_method == 'original':
    X_train_final = X_train_selected
    X_val_final = X_val_selected
elif best_eng_method.startswith('polynomial'):
    X_train_final = best_transformer.transform(X_train_selected)
    X_val_final = best_transformer.transform(X_val_selected)
elif best_eng_method.startswith('pca'):
    X_train_final = best_transformer.transform(X_train_selected)
    X_val_final = best_transformer.transform(X_val_selected)
else:
    X_train_final = X_train_selected
    X_val_final = X_val_selected

print(f"最適化後の特徴量形状: 訓練{X_train_final.shape}, 検証{X_val_final.shape}")
print(f"元の形状: {X_demo_train.shape}")
print(f"最適化手法: {best_eng_method}")
print(f"選択された特徴量数: {best_k}")

# 最適化された特徴量を保存
np.save(os.path.join(output_path, 'X_demographics_train_optimized.npy'), X_train_final)
np.save(os.path.join(output_path, 'X_demographics_val_optimized.npy'), X_val_final)

# 最適化結果を保存
optimization_results = {
    'timestamp': datetime.now().isoformat(),
    'data_info': {
        'original_shape': X_demo_train.shape,
        'optimized_shape': X_train_final.shape,
        'n_classes': len(np.unique(y_train))
    },
    'feature_importance': {
        'top_features': feature_importance.head(10)['feature_idx'].tolist(),
        'feature_scores': feature_importance.to_dict('records')
    },
    'feature_selection': {
        'baseline_score': baseline_mean,
        'best_k': best_k,
        'best_score': best_score,
        'improvement': best_score - baseline_mean,
        'selected_features': best_selector.get_support(indices=True).tolist(),
        'all_results': selection_results
    },
    'feature_engineering': {
        'best_method': best_eng_method,
        'best_score': best_eng_score,
        'improvement': best_eng_score - orig_mean,
        'all_results': engineering_results
    }
}

# JSON形式で保存
results_file = os.path.join(output_path, 'demographics_optimization_results.json')
with open(results_file, 'w', encoding='utf-8') as f:
    json.dump(optimization_results, f, indent=2, ensure_ascii=False)

print(f"\n最適化結果保存完了:")
print(f"- 最適化された特徴量: {output_path}/X_demographics_*_optimized.npy")
print(f"- 最適化結果: {results_file}")

# 最適化結果サマリー
print("\n" + "="*60)
print("Demographics特徴量最適化結果サマリー")
print("="*60)

print(f"📊 データ情報:")
print(f"   - 元の特徴量数: {X_demo_train.shape[1]}")
print(f"   - 最適化後特徴量数: {X_train_final.shape[1]}")
print(f"   - 訓練データ数: {X_train_final.shape[0]}")
print(f"   - 検証データ数: {X_val_final.shape[0]}")
print(f"   - クラス数: {len(np.unique(y_train))}")

print(f"\n🔍 特徴量選択結果:")
print(f"   - ベースライン性能: {baseline_mean:.4f}")
print(f"   - 最適特徴量数: {best_k}")
print(f"   - 選択後性能: {best_score:.4f}")
print(f"   - 改善度: {best_score - baseline_mean:+.4f}")

print(f"\n🛠️ 特徴量エンジニアリング結果:")
print(f"   - 最良手法: {best_eng_method}")
print(f"   - 最良性能: {best_eng_score:.4f}")
print(f"   - 改善度: {best_eng_score - orig_mean:+.4f}")

print(f"\n📈 総合改善度:")
print(f"   - 元の性能: {baseline_mean:.4f}")
print(f"   - 最適化後性能: {best_eng_score:.4f}")
print(f"   - 総合改善度: {best_eng_score - baseline_mean:+.4f}")

print(f"\n✅ 保存ファイル:")
print(f"   - {output_path}/X_demographics_train_optimized.npy")
print(f"   - {output_path}/X_demographics_val_optimized.npy")
print(f"   - {output_path}/demographics_optimization_results.json")

print("="*60)
print("Demographics特徴量最適化完了！")
print("="*60)



=== Trial 0 ===
パラメータ: {'lstm_units_1': 64, 'lstm_units_2': 64, 'dense_units': 56, 'demographics_dense_units': 24, 'fusion_dense_units': 16, 'dropout_rate': 0.1, 'dense_dropout_rate': 0.1, 'learning_rate': 0.005399484409787433, 'batch_size': 32, 'epochs': 20, 'patience': 10, 'reduce_lr_patience': 5, 'use_tqdm': False, 'use_tensorboard': False, 'fusion_type': 'concatenate'}
Mixed precision enabled for better GPU performance
GPU利用可能: 1台
使用GPU: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
LSTM v2学習環境初期化完了
実験名: lstm_v2_hyperopt
ウィンドウ設定: w64_s16
出力ディレクトリ: ../output/experiments/lstm_v2_hyperopt_w64_s16
前処理済みデータ: ../output/experiments/lstm_v2_w64_s16/preprocessed
GPU利用可能: True
前処理済みデータを読み込み中...
最適化されたdemographics特徴量が見つかりません。オリジナルを使用します
データ読み込み完了:
  センサーデータ: (13393, 64, 332)
  Demographics: (13393, 20)
  ラベル: (13393,)
  クラス数: 18
ハイブリッドモデル学習開始 (融合方式: concatenate)...
データ分割中...
テストサイズ: 0.2
検証サイズ: 0.2
データ分割完了:
  訓練 - センサー: (8571, 64, 332), Demographics: (8571, 20), ラベル: (8571,

I0000 00:00:1751864757.791577   99602 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5660 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050, pci bus id: 0000:01:00.0, compute capability: 8.6


GPU上でモデルを作成・学習します
Mixed precision enabled for better GPU performance
GPU利用可能: 1台
使用GPU: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
LSTM v2ハイブリッドモデル初期化完了
センサー入力形状: (64, 332)
Demographics入力形状: (20,)
クラス数: 18
融合方式: concatenate
GPU利用可能: True
LSTM v2ハイブリッドモデルを構築中...
モデル構築完了
総パラメータ数: 141,514

=== モデルサマリー ===


Model: "lstm_v2_hybrid"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ sensor_input        │ (None, 64, 332)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ (None, 64, 64)    │    101,632 │ sensor_input[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_bn_1           │ (None, 64, 64)    │        256 │ lstm_1[0][0]      │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_dropout_1      │ (None, 64, 64)    │          0 │ lstm_bn_1[0][0]   │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_2 (LSTM)       │ (None, 64)        │     33,024 │ lstm_dropout_1[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ demographics_input  │ (None, 20)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_bn_2           │ (None, 64)        │        256 │ lstm_2[0][0]      │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ demographics_dense… │ (None, 24)        │        504 │ demographics_inp… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_dropout_2      │ (None, 64)        │          0 │ lstm_bn_2[0][0]   │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ demographics_dropo… │ (None, 24)        │          0 │ demographics_den… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sensor_dense        │ (None, 56)        │      3,640 │ lstm_dropout_2[0… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ demographics_dense… │ (None, 24)        │        600 │ demographics_dro… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sensor_dropout      │ (None, 56)        │          0 │ sensor_dense[0][… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ demographics_dropo… │ (None, 24)        │          0 │ demographics_den… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ feature_fusion      │ (None, 80)        │          0 │ sensor_dropout[0… │
│ (Concatenate)       │                   │            │ demographics_dro… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ fusion_dense        │ (None, 16)        │      1,296 │ feature_fusion[0… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ fusion_dropout      │ (None, 16)        │          0 │ fusion_dense[0][… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

...

134/134 ━━━━━━━━━━━━━━━━━━━━ 5s 35ms/step - accuracy: 0.2590 - loss: 2.3514 - val_accuracy: 0.2982 - val_loss: 2.1617 - learning_rate: 1.9010e-04
Epoch 11/20
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.2672 - loss: 2.2970
Epoch 11: val_loss improved from 2.16166 to 2.12803, saving model to ../output/experiments/lstm_v2_hyperopt_w64_s16/models/lstm_v2_hybrid_concatenate_best.h5


134/134 ━━━━━━━━━━━━━━━━━━━━ 5s 36ms/step - accuracy: 0.2672 - loss: 2.2969 - val_accuracy: 0.2996 - val_loss: 2.1280 - learning_rate: 1.9010e-04
Epoch 12/20
133/134 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.2766 - loss: 2.2368
Epoch 12: val_loss improved from 2.12803 to 2.08301, saving model to ../output/experiments/lstm_v2_hyperopt_w64_s16/models/lstm_v2_hybrid_concatenate_best.h5


134/134 ━━━━━━━━━━━━━━━━━━━━ 5s 35ms/step - accuracy: 0.2765 - loss: 2.2370 - val_accuracy: 0.3136 - val_loss: 2.0830 - learning_rate: 1.9010e-04
Epoch 13/20
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.2844 - loss: 2.2170
Epoch 13: val_loss improved from 2.08301 to 2.05592, saving model to ../output/experiments/lstm_v2_hyperopt_w64_s16/models/lstm_v2_hybrid_concatenate_best.h5


134/134 ━━━━━━━━━━━━━━━━━━━━ 5s 34ms/step - accuracy: 0.2844 - loss: 2.2169 - val_accuracy: 0.3220 - val_loss: 2.0559 - learning_rate: 1.9010e-04
Epoch 14/20
 54/134 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - accuracy: 0.2989 - loss: 2.1505

[W 2025-07-07 14:11:22,158] Trial 1 failed with parameters: {'lstm_units_1': 48, 'lstm_units_2': 24, 'dense_units': 32, 'demographics_dense_units': 20, 'fusion_dense_units': 32, 'dropout_rate': 0.2, 'dense_dropout_rate': 0.30000000000000004, 'learning_rate': 0.00019010245319870352, 'batch_size': 64, 'fusion_type': 'concatenate'} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/mnt/c/Users/ShunK/works/CMI_comp/.venv/lib/python3.11/site-packages/optuna/study/_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/ipykernel_99602/2320217177.py", line 60, in objective
    results = trainer.train_model(data, model_params, fusion_type)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/mnt/c/Users/ShunK/works/CMI_comp/notebooks/../src/lstm_v2_trainer.py", line 324, in train_model
    history = model.train(
              ^^^^^^^^^^^^
  File "/mnt/c/Users/Shu